# **Final Project - Task 2:** Map reduce for Powerlifting dataset

## **About the dataset** - Vehicle CO2 Emissions Dataset

---

**Context**

This dataset is a snapshot of the `OpenPowerlifting database` as of January 2024. OpenPowerlifting is creating a public-domain archive of powerlifting history. Powerlifting is a sport in which competitors compete to lift the most weight for their class in three separate barbell lifts: the Squat, Bench, and Deadlift.

**Content**

This version of the dataset includs two files: `meets.csv` and `openpowerlifting.csv`.
- `meets.csv` is a record of all meets (competitions) included in the OpenPowerlifting database.
- `openpowerlifting.csv` is a record of all competitors who attended those meets, and the stats and lifts that they recorded at them.


## **Importing the basics**

In [1]:
import pyspark
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


In [2]:
from pyspark.sql import SparkSession

#### **Starting the Spark session**

In [3]:
spark = SparkSession.builder.appName('Final Project Big Data').getOrCreate()

In [4]:
spark

In [5]:
df = spark.sql ("select 'spark' as hello")
df.show(5)

+-----+
|hello|
+-----+
|spark|
+-----+



#### **Mounting the Drive**

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
!ls "drive/MyDrive/Colab Notebooks/FinalProject_BigData"

 co2.csv					 meets.csv
'Copie a fișierului Final_project_task2.ipynb'	 openpowerlifting.csv
 Final_project_task1.ipynb			 Report.gdoc
 Final_project_task2.ipynb


In [8]:
df1 = spark.read.csv('drive/MyDrive/Colab Notebooks/FinalProject_BigData/meets.csv', inferSchema = True, header=True)
df2 = spark.read.csv('drive/MyDrive/Colab Notebooks/FinalProject_BigData/openpowerlifting.csv', inferSchema = True, header=True)

In [9]:
df1.show()

+------+--------------+----------+----------+-----------+---------+-------------+--------------------+
|MeetID|      MeetPath|Federation|      Date|MeetCountry|MeetState|     MeetTown|            MeetName|
+------+--------------+----------+----------+-----------+---------+-------------+--------------------+
|     0|365strong/1601| 365Strong|2016-10-29|        USA|       NC|    Charlotte|2016 Junior & Sen...|
|     1|365strong/1602| 365Strong|2016-11-19|        USA|       MO|        Ozark|Thanksgiving Powe...|
|     2|365strong/1603| 365Strong|2016-07-09|        USA|       NC|    Charlotte|Charlotte Europa ...|
|     3|365strong/1604| 365Strong|2016-06-11|        USA|       SC|    Rock Hill|Carolina Cup Push...|
|     4|365strong/1605| 365Strong|2016-04-10|        USA|       SC|    Rock Hill|Eastern USA Chall...|
|     5|365strong/1701| 365Strong|2017-04-22|        USA|       NC|    Charlotte|Charlotte Europa ...|
|     6|365strong/1702| 365Strong|2017-01-21|        USA|       GA|     M

In [10]:
df2.show()

+------+--------------------+---+----------+----+-----------+------------+-------------+--------+-----------+--------+-----------+-----------+--------------+-------+-----+------+
|MeetID|                Name|Sex| Equipment| Age|   Division|BodyweightKg|WeightClassKg|Squat4Kg|BestSquatKg|Bench4Kg|BestBenchKg|Deadlift4Kg|BestDeadliftKg|TotalKg|Place| Wilks|
+------+--------------------+---+----------+----+-----------+------------+-------------+--------+-----------+--------+-----------+-----------+--------------+-------+-----+------+
|     0|    Angie Belk Terry|  F|     Wraps|47.0|  Mst 45-49|        59.6|           60|    NULL|      47.63|    NULL|      20.41|       NULL|         70.31| 138.35|    1|155.05|
|     0|         Dawn Bogart|  F|Single-ply|42.0|  Mst 40-44|       58.51|           60|    NULL|     142.88|    NULL|      95.25|       NULL|        163.29| 401.42|    1|456.38|
|     0|         Dawn Bogart|  F|Single-ply|42.0|Open Senior|       58.51|           60|    NULL|     142

### **1. Number of participants in each competition**


In [11]:
rdd_participants = df2.rdd

mapped_participants = rdd_participants.map(lambda row: (row['MeetID'], 1))

participants_per_meet = mapped_participants.reduceByKey(lambda a, b: a + b)

print("Number of participants per competition:\n")
for meet_id, participant_count in participants_per_meet.take(20):
    print(f"MeetID {meet_id} : {participant_count} participants")



Number of participants per competition:

MeetID 0 : 79 participants
MeetID 2 : 92 participants
MeetID 4 : 47 participants
MeetID 6 : 41 participants
MeetID 8 : 32 participants
MeetID 10 : 7 participants
MeetID 12 : 128 participants
MeetID 14 : 13 participants
MeetID 16 : 21 participants
MeetID 18 : 16 participants
MeetID 20 : 16 participants
MeetID 22 : 44 participants
MeetID 24 : 66 participants
MeetID 26 : 128 participants
MeetID 28 : 22 participants
MeetID 30 : 13 participants
MeetID 32 : 41 participants
MeetID 34 : 17 participants
MeetID 36 : 24 participants
MeetID 38 : 66 participants


### **2. Competition summary**


In [12]:
rdd_competitions = df1.rdd

mapped_competitions = rdd_competitions.map(lambda row: (row['MeetID'], (row['MeetName'], row['MeetCountry'], row['Date'].year)))

competition_summary_rdd = mapped_competitions.join(participants_per_meet)

competition_summary = competition_summary_rdd.map(lambda x: (x[1][0][0], x[1][0][1], x[1][0][2], x[1][1]))

print("Competition summary:\n")
for name, country, year, count in competition_summary.take(20):
    print(f"{year} | {country:<10} | {count:>4} participants | {name}")


Competition summary:

2016 | USA        |   79 participants | 2016 Junior & Senior National Powerlifting Championships
2016 | USA        |   24 participants | Carolina Cup Push Pull Challenge
2017 | USA        |   41 participants | New Year Challenge
2017 | USA        |   50 participants | Eastern Regional Championship
2017 | USA        |  128 participants | 2017 Men & Women National Powerlifting Championships
2016 | USA        |  211 participants | 2016 AAU Worlds
2017 | USA        |   16 participants | Central California Meet
1995 | USA        |   90 participants | Men's Nationals
2014 | USA        |   66 participants | Elite Fall Classic
2017 | USA        |   29 participants | Rhino Open Championships
2017 | USA        |   13 participants | Missouri Raw Summer Iron Bash
2017 | USA        |   25 participants | Southern States
2017 | USA        |   24 participants | Blessed Iron Bash
2017 | USA        |   17 participants | Region 6 Championships
2017 | USA        |   33 participants |

### **3. Total weight lifted by each participant**

In [13]:
mapped_weight = rdd_participants.map(lambda row: (row['Name'], float(row['TotalKg'] or 0)))

total_weight_per_participant = mapped_weight.reduceByKey(lambda a, b: a + b)

print("\nTotal weight lifted by each participant:\n")
for name, total in total_weight_per_participant.take(20):
    print(f"{name:<30}  |  Total Weight: {total:.2f} kg")




Total weight lifted by each participant:

Angie Belk Terry                |  Total Weight: 138.35 kg
Destiny Dula                    |  Total Weight: 122.47 kg
Courtney Norris                 |  Total Weight: 1802.36 kg
Kayce Hoover                    |  Total Weight: 652.70 kg
Cindy Meeker                    |  Total Weight: 4334.97 kg
Candice Pardue Maness           |  Total Weight: 283.49 kg
Jessica Wissmann                |  Total Weight: 1162.00 kg
Emma Love                       |  Total Weight: 222.25 kg
Kelly Benson                    |  Total Weight: 299.37 kg
Cathy Cranford                  |  Total Weight: 864.99 kg
Alexis Eliopoulos               |  Total Weight: 2037.54 kg
Shannon Nash                    |  Total Weight: 3910.88 kg
Paula Bowers                    |  Total Weight: 215.46 kg
Kevin Gingerich                 |  Total Weight: 2342.57 kg
James McManus                   |  Total Weight: 272.16 kg
Scott Faircloth                 |  Total Weight: 970.69 kg
Brandon